In [18]:
from fz_text_seg import tokenization, lexical_score, getBoundaries

In [19]:
text_tokens, unique_tokens, paragraphs = tokenization('corpus.txt', 6)
text_tokens

[['cat', 'dog', 'appl', 'cat', 'dog', 'dog'],
 ['dog', 'appl', 'cat', 'dog', 'dog', 'cat'],
 ['appl', 'cat', 'dog', 'dog', 'cat'],
 ['cat', 'dog', 'dog', 'cat', 'lot'],
 ['dog', 'dog', 'cat', 'lot'],
 ['dog', 'cat', 'lot', 'cat'],
 ['cat', 'lot', 'cat', 'appl'],
 ['lot', 'cat', 'appl', 'anim'],
 ['lot', 'cat', 'appl', 'anim', 'bird'],
 ['cat', 'appl', 'anim', 'bird', 'anim'],
 ['cat', 'appl', 'anim', 'bird', 'anim', 'appl'],
 ['appl', 'anim', 'bird', 'anim', 'appl', 'banana'],
 ['anim', 'bird', 'anim', 'appl', 'banana', 'kiwi'],
 ['bird', 'anim', 'appl', 'banana', 'kiwi', 'appl'],
 ['anim', 'appl', 'banana', 'kiwi', 'appl', 'cat'],
 ['appl', 'banana', 'kiwi', 'appl', 'cat', 'banana'],
 ['banana', 'kiwi', 'appl', 'cat', 'banana', 'dog'],
 ['kiwi', 'appl', 'cat', 'banana', 'dog', 'bunch'],
 ['appl', 'cat', 'banana', 'dog', 'bunch'],
 ['cat', 'banana', 'dog', 'bunch', 'appl'],
 ['banana', 'dog', 'bunch', 'appl', 'fruit'],
 ['dog', 'bunch', 'appl', 'fruit', 'kiwi'],
 ['bunch', 'appl', 'fru

In [20]:
paragraphs

[69, 142]

In [21]:
lex_score = lexical_score(text_tokens, unique_tokens, 3)
lex_score

[1.0,
 0.8908708063747479,
 0.816496580927726,
 0.6804138174397717,
 0.4082482904638631,
 0.5477225575051661,
 0.4629100498862757,
 0.7905694150420948,
 0.7071067811865475,
 0.8017837257372732,
 0.7826237921249264,
 0.7826237921249264,
 0.5590169943749475,
 0.625,
 0.7216878364870323,
 0.7071067811865475,
 0.7905694150420948,
 0.7302967433402214,
 0.6,
 0.3380617018914066,
 0.3380617018914066,
 0.565685424949238,
 0.5976143046671968,
 0.7559289460184544,
 0.7745966692414834,
 0.6454972243679028,
 0.5,
 0.20412414523193154,
 0.4082482904638631,
 0.75]

In [22]:
getBoundaries(lex_score, paragraphs, 3)

[69]